In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, DecimalType

In [0]:
from databricks.labs.dqx.engine import DQEngine
from databricks.sdk import WorkspaceClient

# Create parameter

In [0]:
dbutils.widgets.text('storage_account', '0')
dbutils.widgets.text('year', '0')
dbutils.widgets.text('month', '0')
dbutils.widgets.text('day', '0')

In [0]:
storage_account = dbutils.widgets.get('storage_account')
year = dbutils.widgets.get('year')
month = dbutils.widgets.get('month')
day = dbutils.widgets.get('day')

# Read data from bronze

In [0]:
df = spark \
        .read \
        .format('parquet') \
        .option('inferSchema', True) \
        .load(f'abfss://bronze@{storage_account}.dfs.core.windows.net/raw_data/{year}/{month}/{day}/')

In [0]:
df.display()

Sales_Person_ID,Sales_Person,Country,Product_ID,Product,Date,Amount,Boxes_Shipped
14,Jehu Rudeforth,UK,15,Mint Chip Choco,2022-01-04,"$5,320",180
24,Van Tuxwell,India,3,85% Dark Bars,2022-08-01,"$7,896",94
10,Gigi Bohling,India,18,Peanut Butter Cubes,2022-07-07,"$4,501",91
13,Jan Morforth,Australia,18,Peanut Butter Cubes,2022-04-27,"$12,726",342
14,Jehu Rudeforth,UK,18,Peanut Butter Cubes,2022-02-24,"$13,685",184
24,Van Tuxwell,India,20,Smooth Sliky Salty,2022-06-06,"$5,376",38
21,Oby Sorrel,UK,4,99% Dark & Pure,2022-01-25,"$13,685",176
11,Gunar Cockshoot,Australia,5,After Nines,2022-03-24,"$3,080",73
14,Jehu Rudeforth,New Zealand,1,50% Dark Bites,2022-04-20,"$3,990",59
4,Brien Boise,Australia,4,99% Dark & Pure,2022-07-04,"$2,835",102


# Transform

### Drop null and duplicate

In [0]:
df = df.dropna() \
        .dropDuplicates(['Sales_Person_ID', 'Country', 'Product_ID', 'Date', 'Boxes_Shipped'])

### Change country name

In [0]:
df = df.withColumn(
        'Country',
        F.when(
            F.col('Country') == 'UK',
            'United Kingdom'
        ).when(
            F.col('Country') == 'USA',
            'United States'
        ).otherwise(F.col('Country'))
    )

In [0]:
df.display()

Sales_Person_ID,Sales_Person,Country,Product_ID,Product,Date,Amount,Boxes_Shipped
16,Karlen McCaffrey,United States,19,Raspberry Choco,2022-04-15,"$14,749",354
24,Van Tuxwell,Australia,14,Milk Bars,2022-06-20,"$7,910",87
6,Ches Bonnell,New Zealand,14,Milk Bars,2022-08-17,"$4,389",126
25,Wilone O'Kielt,Australia,12,Fruit & Nut Bars,2022-06-15,$392,102
23,Roddy Speechley,Canada,16,Orange Choco,2022-02-09,"$8,148",85
18,Madelene Upcott,India,15,Mint Chip Choco,2022-08-24,"$3,836",71
2,Barr Faughny,New Zealand,1,50% Dark Bites,2022-06-23,"$4,557",308
1,Andria Kimpton,United Kingdom,1,50% Dark Bites,2022-08-24,"$2,653",314
24,Van Tuxwell,United States,4,99% Dark & Pure,2022-07-28,"$12,586",6
17,Kelci Walkden,India,5,After Nines,2022-01-07,"$1,687",520


### Remove whitespace

In [0]:
df = df.withColumn('Sales_Person', F.trim('Sales_Person')) \
        .withColumn('Country', F.trim('Country')) \
        .withColumn('Product', F.trim('Product')) \
        .withColumn('Amount', F.trim('Amount'))

In [0]:
df.display()

Sales_Person_ID,Sales_Person,Country,Product_ID,Product,Date,Amount,Boxes_Shipped
16,Karlen McCaffrey,United States,19,Raspberry Choco,2022-04-15,"$14,749",354
24,Van Tuxwell,Australia,14,Milk Bars,2022-06-20,"$7,910",87
6,Ches Bonnell,New Zealand,14,Milk Bars,2022-08-17,"$4,389",126
25,Wilone O'Kielt,Australia,12,Fruit & Nut Bars,2022-06-15,$392,102
23,Roddy Speechley,Canada,16,Orange Choco,2022-02-09,"$8,148",85
18,Madelene Upcott,India,15,Mint Chip Choco,2022-08-24,"$3,836",71
2,Barr Faughny,New Zealand,1,50% Dark Bites,2022-06-23,"$4,557",308
1,Andria Kimpton,United Kingdom,1,50% Dark Bites,2022-08-24,"$2,653",314
24,Van Tuxwell,United States,4,99% Dark & Pure,2022-07-28,"$12,586",6
17,Kelci Walkden,India,5,After Nines,2022-01-07,"$1,687",520


### Change Amount to integer and column name to Revenue

In [0]:
df = df.withColumn('Amount', F.regexp_replace('Amount', '[$,]', '').cast(IntegerType())) \
        .withColumnRenamed('Amount', 'Revenue')

In [0]:
df.display()

Sales_Person_ID,Sales_Person,Country,Product_ID,Product,Date,Revenue,Boxes_Shipped
16,Karlen McCaffrey,United States,19,Raspberry Choco,2022-04-15,14749,354
24,Van Tuxwell,Australia,14,Milk Bars,2022-06-20,7910,87
6,Ches Bonnell,New Zealand,14,Milk Bars,2022-08-17,4389,126
25,Wilone O'Kielt,Australia,12,Fruit & Nut Bars,2022-06-15,392,102
23,Roddy Speechley,Canada,16,Orange Choco,2022-02-09,8148,85
18,Madelene Upcott,India,15,Mint Chip Choco,2022-08-24,3836,71
2,Barr Faughny,New Zealand,1,50% Dark Bites,2022-06-23,4557,308
1,Andria Kimpton,United Kingdom,1,50% Dark Bites,2022-08-24,2653,314
24,Van Tuxwell,United States,4,99% Dark & Pure,2022-07-28,12586,6
17,Kelci Walkden,India,5,After Nines,2022-01-07,1687,520


### Remove records where Revenue or Boxes_Shipped is 0

In [0]:
df = df.filter((df.Revenue != 0) & (df.Boxes_Shipped != 0))

In [0]:
df.display()

Sales_Person_ID,Sales_Person,Country,Product_ID,Product,Date,Revenue,Boxes_Shipped
16,Karlen McCaffrey,United States,19,Raspberry Choco,2022-04-15,14749,354
24,Van Tuxwell,Australia,14,Milk Bars,2022-06-20,7910,87
6,Ches Bonnell,New Zealand,14,Milk Bars,2022-08-17,4389,126
25,Wilone O'Kielt,Australia,12,Fruit & Nut Bars,2022-06-15,392,102
23,Roddy Speechley,Canada,16,Orange Choco,2022-02-09,8148,85
18,Madelene Upcott,India,15,Mint Chip Choco,2022-08-24,3836,71
2,Barr Faughny,New Zealand,1,50% Dark Bites,2022-06-23,4557,308
1,Andria Kimpton,United Kingdom,1,50% Dark Bites,2022-08-24,2653,314
24,Van Tuxwell,United States,4,99% Dark & Pure,2022-07-28,12586,6
17,Kelci Walkden,India,5,After Nines,2022-01-07,1687,520


### Add First_Name, Last_Name columns

In [0]:
df = df.withColumn('First_Name', F.split('Sales_Person', ' ')[0]) \
        .withColumn('Last_Name', F.split('Sales_Person', ' ')[1])

In [0]:
df.display()

Sales_Person_ID,Sales_Person,Country,Product_ID,Product,Date,Revenue,Boxes_Shipped,First_Name,Last_Name
16,Karlen McCaffrey,United States,19,Raspberry Choco,2022-04-15,14749,354,Karlen,McCaffrey
24,Van Tuxwell,Australia,14,Milk Bars,2022-06-20,7910,87,Van,Tuxwell
6,Ches Bonnell,New Zealand,14,Milk Bars,2022-08-17,4389,126,Ches,Bonnell
25,Wilone O'Kielt,Australia,12,Fruit & Nut Bars,2022-06-15,392,102,Wilone,O'Kielt
23,Roddy Speechley,Canada,16,Orange Choco,2022-02-09,8148,85,Roddy,Speechley
18,Madelene Upcott,India,15,Mint Chip Choco,2022-08-24,3836,71,Madelene,Upcott
2,Barr Faughny,New Zealand,1,50% Dark Bites,2022-06-23,4557,308,Barr,Faughny
1,Andria Kimpton,United Kingdom,1,50% Dark Bites,2022-08-24,2653,314,Andria,Kimpton
24,Van Tuxwell,United States,4,99% Dark & Pure,2022-07-28,12586,6,Van,Tuxwell
17,Kelci Walkden,India,5,After Nines,2022-01-07,1687,520,Kelci,Walkden


### Calculate Revenue_Per_Box

In [0]:
df = df.withColumn('Revenue_Per_Box', F.round(df['Revenue'] / df['Boxes_Shipped'], 2).cast(DecimalType(10, 2)))

In [0]:
df.display()

Sales_Person_ID,Sales_Person,Country,Product_ID,Product,Date,Revenue,Boxes_Shipped,First_Name,Last_Name,Revenue_Per_Box
16,Karlen McCaffrey,United States,19,Raspberry Choco,2022-04-15,14749,354,Karlen,McCaffrey,41.66
24,Van Tuxwell,Australia,14,Milk Bars,2022-06-20,7910,87,Van,Tuxwell,90.92
6,Ches Bonnell,New Zealand,14,Milk Bars,2022-08-17,4389,126,Ches,Bonnell,34.83
25,Wilone O'Kielt,Australia,12,Fruit & Nut Bars,2022-06-15,392,102,Wilone,O'Kielt,3.84
23,Roddy Speechley,Canada,16,Orange Choco,2022-02-09,8148,85,Roddy,Speechley,95.86
18,Madelene Upcott,India,15,Mint Chip Choco,2022-08-24,3836,71,Madelene,Upcott,54.03
2,Barr Faughny,New Zealand,1,50% Dark Bites,2022-06-23,4557,308,Barr,Faughny,14.80
1,Andria Kimpton,United Kingdom,1,50% Dark Bites,2022-08-24,2653,314,Andria,Kimpton,8.45
24,Van Tuxwell,United States,4,99% Dark & Pure,2022-07-28,12586,6,Van,Tuxwell,2097.67
17,Kelci Walkden,India,5,After Nines,2022-01-07,1687,520,Kelci,Walkden,3.24


### Extract Date components

In [0]:
df = df.withColumn('Date_Key', F.date_format('Date', 'yyyyMMdd')) \
        .withColumn('Year', F.year('Date')) \
        .withColumn('Quarter', F.quarter('Date')) \
        .withColumn('Month', F.month('Date')) \
        .withColumn('Day', F.dayofmonth('Date')) \
        .withColumn('Start_Of_Year', F.trunc('Date', 'year')) \
        .withColumn('Start_Of_Quarter', F.trunc('Date', 'quarter')) \
        .withColumn('Start_Of_Month', F.trunc('Date', 'month'))

In [0]:
df.display()

Sales_Person_ID,Sales_Person,Country,Product_ID,Product,Date,Revenue,Boxes_Shipped,First_Name,Last_Name,Revenue_Per_Box,Date_Key,Year,Quarter,Month,Day,Start_Of_Year,Start_Of_Quarter,Start_Of_Month
16,Karlen McCaffrey,United States,19,Raspberry Choco,2022-04-15,14749,354,Karlen,McCaffrey,41.66,20220415,2022,2,4,15,2022-01-01,2022-04-01,2022-04-01
24,Van Tuxwell,Australia,14,Milk Bars,2022-06-20,7910,87,Van,Tuxwell,90.92,20220620,2022,2,6,20,2022-01-01,2022-04-01,2022-06-01
6,Ches Bonnell,New Zealand,14,Milk Bars,2022-08-17,4389,126,Ches,Bonnell,34.83,20220817,2022,3,8,17,2022-01-01,2022-07-01,2022-08-01
25,Wilone O'Kielt,Australia,12,Fruit & Nut Bars,2022-06-15,392,102,Wilone,O'Kielt,3.84,20220615,2022,2,6,15,2022-01-01,2022-04-01,2022-06-01
23,Roddy Speechley,Canada,16,Orange Choco,2022-02-09,8148,85,Roddy,Speechley,95.86,20220209,2022,1,2,9,2022-01-01,2022-01-01,2022-02-01
18,Madelene Upcott,India,15,Mint Chip Choco,2022-08-24,3836,71,Madelene,Upcott,54.03,20220824,2022,3,8,24,2022-01-01,2022-07-01,2022-08-01
2,Barr Faughny,New Zealand,1,50% Dark Bites,2022-06-23,4557,308,Barr,Faughny,14.80,20220623,2022,2,6,23,2022-01-01,2022-04-01,2022-06-01
1,Andria Kimpton,United Kingdom,1,50% Dark Bites,2022-08-24,2653,314,Andria,Kimpton,8.45,20220824,2022,3,8,24,2022-01-01,2022-07-01,2022-08-01
24,Van Tuxwell,United States,4,99% Dark & Pure,2022-07-28,12586,6,Van,Tuxwell,2097.67,20220728,2022,3,7,28,2022-01-01,2022-07-01,2022-07-01
17,Kelci Walkden,India,5,After Nines,2022-01-07,1687,520,Kelci,Walkden,3.24,20220107,2022,1,1,7,2022-01-01,2022-01-01,2022-01-01


# Data quality checks

### Initialize data quality engine

In [0]:
dq_engine = DQEngine(WorkspaceClient())

### Get check conditions from yaml file in workspace

In [0]:
checks = dq_engine.load_checks_from_workspace_file(workspace_path='/pipeline_project/check/checks_silver.yml')

### Apply checks to dataframe

In [0]:
df_check = dq_engine.apply_checks_by_metadata(df, checks)

In [0]:
df_check.display()

Sales_Person_ID,Sales_Person,Country,Product_ID,Product,Date,Revenue,Boxes_Shipped,First_Name,Last_Name,Revenue_Per_Box,Date_Key,Year,Quarter,Month,Day,Start_Of_Year,Start_Of_Quarter,Start_Of_Month,_errors,_warnings
16,Karlen McCaffrey,United States,19,Raspberry Choco,2022-04-15,14749,354,Karlen,McCaffrey,41.66,20220415,2022,2,4,15,2022-01-01,2022-04-01,2022-04-01,null,null
24,Van Tuxwell,Australia,14,Milk Bars,2022-06-20,7910,87,Van,Tuxwell,90.92,20220620,2022,2,6,20,2022-01-01,2022-04-01,2022-06-01,null,null
6,Ches Bonnell,New Zealand,14,Milk Bars,2022-08-17,4389,126,Ches,Bonnell,34.83,20220817,2022,3,8,17,2022-01-01,2022-07-01,2022-08-01,null,null
25,Wilone O'Kielt,Australia,12,Fruit & Nut Bars,2022-06-15,392,102,Wilone,O'Kielt,3.84,20220615,2022,2,6,15,2022-01-01,2022-04-01,2022-06-01,null,null
23,Roddy Speechley,Canada,16,Orange Choco,2022-02-09,8148,85,Roddy,Speechley,95.86,20220209,2022,1,2,9,2022-01-01,2022-01-01,2022-02-01,null,null
18,Madelene Upcott,India,15,Mint Chip Choco,2022-08-24,3836,71,Madelene,Upcott,54.03,20220824,2022,3,8,24,2022-01-01,2022-07-01,2022-08-01,null,null
2,Barr Faughny,New Zealand,1,50% Dark Bites,2022-06-23,4557,308,Barr,Faughny,14.80,20220623,2022,2,6,23,2022-01-01,2022-04-01,2022-06-01,null,null
1,Andria Kimpton,United Kingdom,1,50% Dark Bites,2022-08-24,2653,314,Andria,Kimpton,8.45,20220824,2022,3,8,24,2022-01-01,2022-07-01,2022-08-01,null,null
24,Van Tuxwell,United States,4,99% Dark & Pure,2022-07-28,12586,6,Van,Tuxwell,2097.67,20220728,2022,3,7,28,2022-01-01,2022-07-01,2022-07-01,null,null
17,Kelci Walkden,India,5,After Nines,2022-01-07,1687,520,Kelci,Walkden,3.24,20220107,2022,1,1,7,2022-01-01,2022-01-01,2022-01-01,null,null


### Check error result

In [0]:
error_count = df_check.select('_errors').filter(F.col('_errors').isNotNull()).count()

assert error_count == 0, f'{error_count} errors found in the data'

# Write data to silver

In [0]:
df.write \
    .format('parquet') \
    .mode('overwrite') \
    .save(f'abfss://silver@{storage_account}.dfs.core.windows.net/transformed_data/{year}/{month}/{day}/')